In [80]:
import pandas as pd
import regex as re

pd.set_option('display.max_columns', None)

In [10]:
path_raw = '/home/matteo@COPPET/Documents/data_science/projects/housing_prices_firenze/data/raw/'

In [135]:
caratteristiche = pd.read_excel(path_raw+'caratteristiche.xlsx')
costi = pd.read_excel(path_raw+'costi.xlsx')
efficienza_energetica = pd.read_excel(path_raw+'efficienza_energetica.xlsx')

In [136]:
caratteristiche.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9059 entries, 0 to 9058
Data columns (total 17 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   immobile garantito                            3826 non-null   object 
 1   riferimento e Data annuncio                   9059 non-null   object 
 2   contratto                                     9059 non-null   object 
 3   tipologia                                     9059 non-null   object 
 4   superficie                                    9051 non-null   object 
 5   locali                                        9043 non-null   object 
 6   piano                                         8855 non-null   object 
 7   totale piani edificio                         8361 non-null   object 
 8   disponibilità                                 8078 non-null   object 
 9   Tipo proprietà                                8575 non-null   o

In [137]:
costi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9059 entries, 0 to 9058
Data columns (total 13 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   prezzo                                 9059 non-null   object 
 1   informazioni catastali                 5787 non-null   object 
 2   spese condominio                       5254 non-null   object 
 3   stato                                  0 non-null      float64
 4   certificazione energetica              0 non-null      float64
 5   Efficienza energetica                  0 non-null      float64
 6   Tipo vendita                           0 non-null      float64
 7   data vendita                           0 non-null      float64
 8   riscaldamento                          0 non-null      float64
 9   anno di costruzione                    0 non-null      float64
 10  Prestazione energetica del fabbricato  0 non-null      float64
 11  Clim

In [138]:
efficienza_energetica.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9059 entries, 0 to 9058
Data columns (total 15 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   anno di costruzione                    6560 non-null   float64
 1   stato                                  8795 non-null   object 
 2   riscaldamento                          8631 non-null   object 
 3   Climatizzazione                        4936 non-null   object 
 4   Indice prest. energetica rinnovabile   287 non-null    object 
 5   Prestazione energetica del fabbricato  596 non-null    object 
 6   Efficienza energetica                  7812 non-null   object 
 7   certificazione energetica              1363 non-null   object 
 8   numero immobili                        0 non-null      float64
 9   offerta minima                         0 non-null      float64
 10  rialzo minimo                          0 non-null      float64
 11  Spes

In [139]:
# Drop NaN columns (based on info above, possible error in scraping)
caratteristiche = caratteristiche.drop(columns=['unità', 'Data di inizio lavori e di consegna prevista', 'Dati catastali'])
costi = costi.iloc[:, :3]
efficienza_energetica = efficienza_energetica.iloc[:, :8]

In [140]:
efficienza_energetica.head()

,anno di costruzione,stato,riscaldamento,Climatizzazione,Indice prest. energetica rinnovabile,Prestazione energetica del fabbricato,Efficienza energetica,certificazione energetica
0,1960.0,Ottimo / Ristrutturato,"Autonomo, a radiatori, alimentato a metano","Autonomo, freddo/caldo",NaN,NaN,"E 175,00 kWh/m² anno",NaN
1,1500.0,Buono / Abitabile,"Autonomo, a radiatori, alimentazione elettrica",NaN,NaN,NaN,NaN,Esente
2,1902.0,NaN,"Autonomo, a radiatori, alimentato a metano",NaN,NaN,NaN,In attesa di certificazione,"G 342,60 kWh/m² anno"
3,1500.0,Buono / Abitabile,"Autonomo, a radiatori, alimentato a metano",NaN,NaN,NaN,NaN,Non classificabile
4,1980.0,Buono / Abitabile,"Centralizzato, a radiatori",NaN,NaN,invernale estivo,G,NaN


In [151]:
# Join tables
df = caratteristiche.join(costi).join(efficienza_energetica)

In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9059 entries, 0 to 9058
Data columns (total 25 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   immobile garantito                     3826 non-null   object 
 1   riferimento e Data annuncio            9059 non-null   object 
 2   contratto                              9059 non-null   object 
 3   tipologia                              9059 non-null   object 
 4   superficie                             9051 non-null   object 
 5   locali                                 9043 non-null   object 
 6   piano                                  8855 non-null   object 
 7   totale piani edificio                  8361 non-null   object 
 8   disponibilità                          8078 non-null   object 
 9   Tipo proprietà                         8575 non-null   object 
 10  altre caratteristiche                  8477 non-null   object 
 11  Post

In [143]:
df.head()

,immobile garantito,riferimento e Data annuncio,contratto,tipologia,superficie,locali,piano,totale piani edificio,disponibilità,Tipo proprietà,altre caratteristiche,Posti Auto,indirizzo,zona,prezzo,informazioni catastali,spese condominio,anno di costruzione,stato,riscaldamento,Climatizzazione,Indice prest. energetica rinnovabile,Prestazione energetica del fabbricato,Efficienza energetica,certificazione energetica
0,"Dati certificabili, documentazione completa ...",3v ghirlandaio - 03/10/2020,Vendita,Appartamento,73 m²,"3 (2 camere da letto, 1 altro), 2 bagni, cucin...",3° piano,3 piani,Libero,"Intera proprietà, classe immobile signorile",Fibra ottica\n \n\n ...,NaN,"['Firenze', 'Via del Ghirlandaio, 58', 'Via de...",campo-di-marte-liberta/,€ 295.000,€ 20/mese,"Classe A/2, rendita € 800",1960.0,Ottimo / Ristrutturato,"Autonomo, a radiatori, alimentato a metano","Autonomo, freddo/caldo",NaN,NaN,"E 175,00 kWh/m² anno",NaN
1,"Dati certificabili, documentazione completa ...",F200167 - 03/10/2020,Vendita,Open space,20 m²,"1 locale, 1 bagno, cucina angolo cottura",2° piano,5 piani,Libero,"Intera proprietà, classe immobile signorile",Caminetto\n \n\n ...,NaN,"['Firenze', 'Piazza della Signoria', 'Signoria...",centro/,€ 115.000,€ 20/mese,"Classe C/2, rendita € 1",1500.0,Buono / Abitabile,"Autonomo, a radiatori, alimentazione elettrica",NaN,NaN,NaN,NaN,Esente
2,NaN,T10763_10 - 02/10/2020,Vendita,Terratetto unifamiliare,300 m²,"5+ (4 camere da letto, 5 altri), 2 bagni, cuci...",Piano rialzato,4 piani,2 in garage/box,Libero,"Intera proprietà, classe immobile signorile",Caminetto\n \n\n ...,"['Firenze', 'Castello', 'via giuseppe parini']",serpiolle-careggi/,€ 650.000,NaN,NaN,1902.0,NaN,"Autonomo, a radiatori, alimentato a metano",NaN,NaN,NaN,In attesa di certificazione,"G 342,60 kWh/m² anno"
3,NaN,37551014-27 - 02/10/2020,Vendita,Appartamento,200 m²,"5+ (4 camere da letto, 4 altri), 3 bagni, cuci...","2° piano, con ascensore",2 piani,Libero,"Intera proprietà, classe immobile media",Terrazza\n \n\n ...,NaN,"['Firenze', 'Via Maggio', 'Ponte Vecchio - Pit...",oltrarno/,€ 1.150.000,NaN,NaN,1500.0,Buono / Abitabile,"Autonomo, a radiatori, alimentato a metano",NaN,NaN,NaN,NaN,Non classificabile
4,NaN,33201042-1 - 02/10/2020,Vendita,Appartamento,58 m²,"3 (1 camera da letto, 2 altri), 1 bagno, cucin...","1° piano, con ascensore",4 piani,Libero,"Intera proprietà, classe immobile media",Balcone\n \n\n ...,NaN,"['Firenze', 'Castello', 'via Giulio Bechi']",serpiolle-careggi/,€ 199.000,€ 80/mese,Rendita € 588,1980.0,Buono / Abitabile,"Centralizzato, a radiatori",NaN,NaN,invernale estivo,G,NaN


In [152]:
df.columns = map(str.capitalize, df.columns)
df = df.rename(columns={'Superficie': 'Superficie [m2]', 'Prezzo': 'Prezzo [EUR]'})

In [153]:
df = df.loc[df['Contratto'] != 'Affitto', :]

In [154]:
# Clean up data types
df['Superficie [m2]'] = df['Superficie [m2]'].str.split('m²').str[0].astype('float')

df['Totale piani edificio'] = df['Totale piani edificio'].str.split(' ').str[0]

df['Zona'] = (df['Zona']
              .str.replace('-', ' ')
              .str.replace('/', '')
              .str.title())

df['Prezzo [EUR]'] = df['Prezzo [EUR]'].str.split('€').str[1].str.replace('.', '').astype('float')

In [155]:
df['Prezzo [EUR]'].mean()

489056.3424204403

In [156]:
df['Riscaldamento'].unique()

array(['Autonomo, a radiatori, alimentato a metano',
       'Autonomo, a radiatori, alimentazione elettrica',
       'Centralizzato, a radiatori', 'Autonomo, a radiatori', 'Autonomo',
       'Centralizzato, a radiatori, alimentato a metano', nan,
       'Autonomo, alimentato a gas',
       'Centralizzato, a radiatori, alimentato a gas',
       'Centralizzato, a pavimento, alimentato a fotovoltaico',
       'Centralizzato, a pavimento, alimentato a gas',
       'Autonomo, a radiatori, alimentato a gas',
       'Centralizzato, alimentato a metano', 'Autonomo, ad aria',
       'Autonomo, a stufa', 'Centralizzato, a pavimento', 'Centralizzato',
       'Autonomo, a pavimento, alimentato a metano',
       'Centralizzato, a pavimento, alimentato a metano',
       'Autonomo, ad aria, alimentato a pompa di calore',
       'Autonomo, a pavimento, alimentato a gas',
       'Autonomo, ad aria, alimentato a gas',
       'Autonomo, a pavimento, alimentazione elettrica',
       'Autonomo, a radiatori

In [157]:
# Create type of heating feature
df['Tipologia di riscaldamento'] = df['Riscaldamento'].str.split(',').str[0]

In [167]:
# Checking for and removing duplicates
df.duplicated().sum()
df = df.drop_duplicates()

In [169]:
# Dropping N/A values
df.head()

,Immobile garantito,Riferimento e data annuncio,Contratto,Tipologia,Superficie [m2],Locali,Piano,Totale piani edificio,Disponibilità,Tipo proprietà,Altre caratteristiche,Posti auto,Indirizzo,Zona,Prezzo [EUR],Informazioni catastali,Spese condominio,Anno di costruzione,Stato,Riscaldamento,Climatizzazione,Indice prest. energetica rinnovabile,Prestazione energetica del fabbricato,Efficienza energetica,Certificazione energetica,Tipologia di riscaldamento
0,"Dati certificabili, documentazione completa ...",3v ghirlandaio - 03/10/2020,Vendita,Appartamento,73.0,"3 (2 camere da letto, 1 altro), 2 bagni, cucin...",3° piano,3,Libero,"Intera proprietà, classe immobile signorile",Fibra ottica\n \n\n ...,NaN,"['Firenze', 'Via del Ghirlandaio, 58', 'Via de...",Campo Di Marte Liberta,295000.0,€ 20/mese,"Classe A/2, rendita € 800",1960.0,Ottimo / Ristrutturato,"Autonomo, a radiatori, alimentato a metano","Autonomo, freddo/caldo",NaN,NaN,"E 175,00 kWh/m² anno",NaN,Autonomo
1,"Dati certificabili, documentazione completa ...",F200167 - 03/10/2020,Vendita,Open space,20.0,"1 locale, 1 bagno, cucina angolo cottura",2° piano,5,Libero,"Intera proprietà, classe immobile signorile",Caminetto\n \n\n ...,NaN,"['Firenze', 'Piazza della Signoria', 'Signoria...",Centro,115000.0,€ 20/mese,"Classe C/2, rendita € 1",1500.0,Buono / Abitabile,"Autonomo, a radiatori, alimentazione elettrica",NaN,NaN,NaN,NaN,Esente,Autonomo
2,NaN,T10763_10 - 02/10/2020,Vendita,Terratetto unifamiliare,300.0,"5+ (4 camere da letto, 5 altri), 2 bagni, cuci...",Piano rialzato,4,2 in garage/box,Libero,"Intera proprietà, classe immobile signorile",Caminetto\n \n\n ...,"['Firenze', 'Castello', 'via giuseppe parini']",Serpiolle Careggi,650000.0,NaN,NaN,1902.0,NaN,"Autonomo, a radiatori, alimentato a metano",NaN,NaN,NaN,In attesa di certificazione,"G 342,60 kWh/m² anno",Autonomo
3,NaN,37551014-27 - 02/10/2020,Vendita,Appartamento,200.0,"5+ (4 camere da letto, 4 altri), 3 bagni, cuci...","2° piano, con ascensore",2,Libero,"Intera proprietà, classe immobile media",Terrazza\n \n\n ...,NaN,"['Firenze', 'Via Maggio', 'Ponte Vecchio - Pit...",Oltrarno,1150000.0,NaN,NaN,1500.0,Buono / Abitabile,"Autonomo, a radiatori, alimentato a metano",NaN,NaN,NaN,NaN,Non classificabile,Autonomo
4,NaN,33201042-1 - 02/10/2020,Vendita,Appartamento,58.0,"3 (1 camera da letto, 2 altri), 1 bagno, cucin...","1° piano, con ascensore",4,Libero,"Intera proprietà, classe immobile media",Balcone\n \n\n ...,NaN,"['Firenze', 'Castello', 'via Giulio Bechi']",Serpiolle Careggi,199000.0,€ 80/mese,Rendita € 588,1980.0,Buono / Abitabile,"Centralizzato, a radiatori",NaN,NaN,invernale estivo,G,NaN,Centralizzato
